# SMPC Protocols Explanation 
In this notebook I'll will give an overview and a quick explanation over the different SMPC protocols that are currently implemented in PySyft. I'll also elaborate on when their applicability to different use-cases (namely different computations) and compare their performance compared to each other and compared to the non-encrypted computations. 


## Quick recap - What is SMPC encryption?
As a quick recap "SMPC" or "MPC" stands for **Multi-Party Computation** and constitutes a form of encryption which can be used for Machine Learning (i.e. it is possible to do calculations on encrypted data) leveraging a network of min. 2 different servers. These are being trusted of not collaborating with each other and as a whole function as a **trusted execution environment** on which sensitive calculations such as model inference, training, etc. can be done without the model or the data being disclosed to any party besides the owner. 

## General concepts 
* Explain in brief important concepts that all protocols leverage 
  * crypto_store 
  * ...

---
--> START WITH LESS PERFORMANT PROTOCOL

In [ ]:
# Setup of example use-case problem 
# Serves to compare best models that can still be encrypted using the different protocols +
# Comparison to non-encrypted computation. (Only inference, we use pre-trained models)

# 1. SPDZ Protocol 
* Paper + Code
* Introduction of first protocol along quick intro into beaver triples 
* Basis for advanced protocols secureNN and FSS. 

### General concept 
There are two types of SMPC: secret-sharing-based SMPC and circuit-gabling-based SMPC. 
The two protocols (secureNN and FSS) that are implemented in PySyft to date are both based on the **SPDZ protocol**, 
which is based on *additive secret-sharing* (the first category) For more information on what that is check out the tutorials by Andrew Trask as part of the Udacity Private and Secure AI Course.<br>
SPDZ is a very widely used protocol for linear functions, meaning it can be used to compute sums and multiplication of encrypted variables. Its multiplication is done using so called **"beaver multiplication triples"** which make the computation very efficient compared to other appraoches (e.g. garbled circuits)
<br>
<br>
The high-level procedure of SPDZ-based protocols is as follow: (based on definition from [bristol crypto blog](https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html))
   * Parties secret-share their inputs at beginning (One crypto-provider exists to generate beaver triples, etc.)
   * Parties compute mul. and sum. on locally and results don't need to be explicitly shared after computation because each participating server already has them ---> <span style="color: red;">**CHECK HOW**</span>
   * At the end parties "open" their calcultaion by sharing their results by sending their share to all others and adding them all parts up (each server)

### Extra: Beaver Multiplication Triples
* Understand high level how triples are used for an efficient masking...(what is simpler alternative and how can this be generalized for other use-cases?) **Only high-level understanding is relevant!**
<br> 

<span style="color:red;">***TODO:*** Mainly summarize **very briefly** why we need those and how we use them. Use Motern Dahl's Blog Bost and bristol crypto blog as reference.</span>

### Extra: Garbled Circuits 
<span style="color:red;">***TODO:*** Quick explanation what that is, where it is used, etc.</span>


* *In general:*
  * There are boolean circuits and arithmetic circuits (MatMul, etc.), both can be garbled (although commonly garbled circuits mean garbled boolean circuits)

# 2. SecureNN Protocol
* Paper + Code ***ADD SOURCE!!***
* Intro to Garbled Circuits (or even earlier)

## High-Level concept 
SecureNN uses the SPDZ protocol for linear layers (beaver triples, etc.) and contains multiple efficent protocols for common non-linearities, as further described below. Compared to ealier work SecureNN implements the computation of the non-linearities without the need of Garbled Circuits (as is the case for SecureML, which was considered the state-of-the-art ML protocol before SecureNN) This also means "interconversation protocols" to bridge between encoding needed for SPDZ and encoding needed for Garbled Circuits aren't necessary. \n 
*In general SecureNN is therefore a lot faster than SecureML and other garbled-circuit-based (for non-linearities) protocols.* 

<span style="color:red;">***TODO:*** Last sentence ONLY FOR NOW.</span>

## In detail 
### Possible computations - Low Level 
This isn't a general purpose protocol, which can compute all possible kinds of computations that are used in training NNs, but by giving up this flexibility (e.g. no usasge of garbled circuits) we gain efficiency which is a vital criteria for the real-world-applicability of an encryption protocol. 

* Matrix Multiplication 
* Select Share 
* Private Compare 
* Share Convert 
* Compute MSB (Most-Significant-Bit)

<span style="color:red;">***TODO:*** Explain in brief what they do and what they are used for (concerning below list of high-level functions)</span>

### Possible functions - High Level (Non exhaustive list)
Resulting from the possible low level computations the following standard models, optimizers and loss-functions can be used. *This should provide a useful summary, but doesn't claim to be an exhaustive list (feel free to add important items that can be composed out of the above mentioned possible low-level computations)

* **Model Architecture:** Includes the possibility for encrypted computation of the derivative (needed for backprop)
    * **Linear Layers** with SPDZ using beaver triples 
      * Matrix Multiplication and Convolutions (in CNNs)
      * Average Pooling 
      * Normalization (as Division by a shared value is possible)
    * **Non-Linear Layers:** 
      * Max Pooling 
      * ReLU, Leaky ReLU, Piece-wise linear activation functions 
     
* **Optimizers:**
  * SGD with Momentum 
  * ADAM?

* **Loss-functions:**
  * Cross-Entropy-Loss
  
<span style="color:red;">***TODO:*** Check if most used optims and loss-functions are compatible.</span>

### Security Guarantees 
* Full Security includes **privacy and correctness**
* The following to gurantees hold for all settings where there is a majority of honest participating servers (**Not in dishonest majority setting!**)
1. **Full Security for semi-honest corruption of a server** 
  * Privacy and Correctness of the data is secured if a server is being *honest-but-curious.* Meaning that follows the protocol but tries to infer as much information about the data it sees as possible. 
2. **Privacy against malicious server** 
  * Even a server which doesn't follow the given protocol can't learn anything about the inputs and outputs of the other (honest) servers 


### Performance Evaluation - Important Metrics 
* Important Metrics: 
  * Communication Complexity 
  * Round Complexity 
  
*See picture from SecureNN paper. "The function
Linearm,n,v denotes a matrix multiplication of dimen-
sion m × n with n × v. Conv2dm,i,f,o denotes a convo-
lutional layer with input m × m, i input channels, a
filter of size f × f, and o output channels. lD denotes
precision of bits. Maxpooln and DMPn denotes Maxpool
and its derivative over n elements. All commu- nication is measured for l−bit inputs and p 
denotes the field size (which is 67 in our case)"*

<span style="color:red;">***TODO:*** Do very small comparision with other existing protocols  + try to evaluate performance based on below use-case. More detailed comparison between secureNN and FSS (AriaNN implementation) and non-encrypted scenario at the end!</span><br><br>
<span style="color:red;">***TODO:*** Think whether Info-text and complexity table are helpful (Table 2)</span>

In [ ]:
# Example Implementation 

## Functional Secret Sharing Protocol

### AriaNN Implementation 
* Paper + Code
---
* **Structure:**
  * Theoretical Concept: Procedure, Safety Guarantees (Practical examples)
  * Theoretical Capabilities in terms of what computations are possible in encrypted form
  * *Concerning PySyft Implementation:* 
    * What was implemented, how? 
    * **MOST IMPORTANT / KEY QUESTION:** For what kind of computations, i.e. models, and in what kind of scenarios can I use this, i.e. what privacy and safety guarantees does this protocol provide in the current implementation. 
---
* **Code:**
  * Try to find most advanced model compatible encryptable with this approach
  * Do simple evaluation of most important metrics (possibly comparing with and without encryption)
    * Time 
    * ...

# Other encryption protocols in PySyft 

## FALCON Protocol (Future)
* Paper + Code 

## Temporary Notes: 
Important Concepts: 
--> Source: [The three part blog series about SPDZ](https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html) and [Morten Dahl's Blogs on SPDZ](https://mortendahl.github.io/2017/09/03/the-spdz-protocol-part1/)
* Functions are often refered to as circuits -> every function can be approx. represented by nested boolean circuits (combination of boolean gates) ??
* There are two types of SMPC: secret-sharing-based SMPC and circuit-gabling-based SMPC 
    * Basic protocol steps for secret-sharing-based SMPC see BLOG: https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html
    * Secret-sharing is mainly done via so called "Additive Secret Sharing" protocol, **Shamir's protocol** is a more complicated secret-sharing technique that doesn't only depend on additive splitting (reconstructable with a simple sum)
* SPDZ is a prominent example of secret-sharing-based SMPC 
  * It is considered to be the optimal method to use FHE in MPC according to authors of the original paper - as described in the blog(for the preprocessing)
  * The online phase is very good already (**TODO:** enumerate number of communications based on blog)
  * That's why mainly the offline phase - i.a. the preprocessing, generation of beaver triples and the big random numbers to serve to secret share values (the party having the real value also participates, other protocol than so far in tutorials where two dedicated units were used) - is being optimized. 
  * In the original SPDZ the preprocessing is done using Somewhat Homomorphic Encryption (SHE), for now no method exists to use instead FHE (**TODO:** How exactly? Are triples different for different functions?) in a competitive manner compared to other MPC protocols. 
  
--- 
 
--> Performance Metrics of Crypto-Protocols: Source [Morten Dahl's Blog on Private Image Analysis with MPC](https://mortendahl.github.io/2017/09/19/private-image-analysis-with-mpc/) 
* Communication Complexity: How many bits are being exchanged between participating parties? 
* Round Complexity: How many points of synchronization are needed in the protocol? (The more the likelier that one party is IDLE because it needs to wait for other party to finish some job e.g.)

---